In [1]:
cd ../analysis/

/home/jlee782/hcomp_paper_seg/analysis


In [2]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [7]:
from collections import defaultdict
import csv

all_data = defaultdict(lambda: defaultdict(lambda: defaultdict(dict)))
# all_data[nworkers][sample_num][objid][clust]
with open('pixel_em/all_PRJ_table.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        nworkers = int(row['num_workers'])
        sample_num = int(row['sample_num'])
        objid = int(row['objid'])
        clust = int(row['clust'])
#         print nworkers, sample_num, objid, clust
        all_data[nworkers][sample_num][objid][clust] = row

print all_data[5][0][1].keys()

[0, 1]


In [8]:
best_clust = defaultdict(lambda: defaultdict(dict))  # best_clust[nworkers][sample_num][objid] = clust
with open('best_clust_picking.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        nworkers = int(row['num_workers'])
        sample_num = int(row['sample_num'])
        objid = int(row['objid'])
        clust = int(row['clust'])
#         print nworkers, sample_num, objid, clust
        best_clust[nworkers][sample_num][objid] = clust

In [10]:
########## Checking (nworkers, sample, objid) missing -1 clust and with best clust = -1 #########
obj_missing_noclust = []
obj_with_noclust_best = []
for nworkers in all_data.keys():
    for sample_num in all_data[nworkers]:
        for objid in all_data[nworkers][sample_num]:
            if -1 not in all_data[nworkers][sample_num][objid].keys():
#                 print '({}, {}, {})'.format(nworkers, sample_num, objid)
                obj_missing_noclust.append((nworkers, sample_num, objid))
            if best_clust[nworkers][sample_num][objid] == -1:
                obj_with_noclust_best.append((nworkers, sample_num, objid))
print 'num (sample x obj) missing clust = -1 in all_data:', len(obj_missing_noclust)
print 'num (sample x obj) with best_clust = -1', len(obj_with_noclust_best)

num (sample x obj) missing clust = -1 in all_data: 775
num (sample x obj) with best_clust = -1 827


In [11]:
########## comparing MV jaccs for noclust vs bestclust
jacc_noclust = defaultdict(list)  # jacc[nworkers] = []
jacc_bestclust = defaultdict(list)
for nworkers in all_data.keys():
    for sample_num in all_data[nworkers]:
        for objid in all_data[nworkers][sample_num]:
            best = best_clust[nworkers][sample_num][objid]
            if best == -1 or -1 not in all_data[nworkers][sample_num][objid].keys():
                print best, all_data[nworkers][sample_num][objid].keys()
                continue
            noclust_MV = all_data[nworkers][sample_num][objid][-1]['J [MV]']
            print nworkers
            jacc_noclust[nworkers].append(noclust_MV)
            bestclust_MV = all_data[nworkers][sample_num][objid][best]['J [MV]']
            jacc_bestclust[nworkers].append(bestclust_MV)

x = []
y_noclust = []
y_bestclust = []
for nworkers in jacc_noclust.keys():
    x.append(nworkers)
    y_noclust.append(np.mean(jacc_noclust[nworkers]))
    y_bestclust.append(np.mean(jacc_bestclust[nworkers]))

print x

0 [0, 1]
-1 [-1]
-1 [-1]
0 [0, 1]
-1 [-1]
-1 [-1]
0 [0, 1]
0 [0, 1, 2]
-1 [-1]
0 [0, 1]
-1 [-1]
-1 [-1]
-1 [-1]
-1 [-1]
0 [0, 1]
-1 [-1]
-1 [-1]
0 [0, 1]
-1 [-1]
1 [0, 1, 2, 3, 4]
0 [0, 1]
0 [0, 1]
-1 [-1]
-1 [-1]
-1 [0, 1]
0 [0, 1]
0 [0, 1, 2, 3]
0 [0, 1]
-1 [0, 1, 2]
1 [0, 1]
-1 [0, 1, 2]
0 [0, 1]
0 [0, 1]
1 [0, 1]
-1 [-1]
1 [0, 1]
-1 [-1]
-1 [-1]
-1 [0, 1]
0 [0, 1]
0 [0, 1]
-1 [-1]
-1 [-1]
-1 [-1]
-1 [-1]
-1 [0, 1, 2]
0 [0, 1]
-1 [-1]
-1 [-1]
0 [0, 1]
-1 [-1]
-1 [-1]
-1 [0, 1]
0 [0, 1, 2]
-1 [-1]
0 [0, 1]
-1 [-1]
-1 [-1]
-1 [-1]
-1 [-1]
0 [0, 1]
-1 [-1]
-1 [-1]
0 [0, 1]
-1 [-1]
2 [0, 1, 2, 3, 4]
0 [0, 1]
0 [0, 1]
-1 [-1]
-1 [-1]
-1 [0, 1]
0 [0, 1]
0 [0, 1, 2, 3]
0 [0, 1]
1 [0, 1, 2]
1 [0, 1]
0 [0, 1, 2]
0 [0, 1]
0 [0, 1]
1 [0, 1]
-1 [-1]
1 [0, 1]
-1 [-1]
-1 [-1]
-1 [0, 1]
-1 [0, 1]
0 [0, 1]
-1 [-1]
-1 [-1]
-1 [-1]
-1 [-1]
0 [0, 1, 2]
0 [0, 1]
-1 [-1]
-1 [-1]
0 [0, 1]
-1 [-1]
-1 [-1]
1 [0, 1]
0 [0, 1, 2]
-1 [-1]
0 [0, 1]
-1 [-1]
-1 [-1]
-1 [-1]
-1 [-1]
0 [0, 1]
-1 [-1]
-1 [-1]
0 [0, 

-1 [-1]
-1 [-1]
-1 [0, 1]
-1 [0, 1]
0 [0, 1]
-1 [-1]
-1 [-1]
-1 [-1]
-1 [-1]
0 [0, 1, 2]
0 [0, 1]
-1 [-1]
-1 [-1]
-1 [0, 1]
-1 [-1]
-1 [-1]
1 [0, 1]
-1 [0, 1, 2]
-1 [-1]
-1 [0, 1]
-1 [-1]
-1 [-1]
-1 [-1]
-1 [-1]
0 [0, 1]
-1 [-1]
-1 [-1]
0 [0, 1]
-1 [-1]
1 [0, 1, 2, 3, 4]
0 [0, 1]
-1 [0, 1]
-1 [-1]
-1 [-1]
1 [0, 1]
0 [0, 1]
0 [0, 1, 2, 3]
0 [0, 1]
0 [0, 1, 2]
1 [0, 1]
0 [0, 1, 2]
-1 [0, 1]
0 [0, 1]
-1 [0, 1]
-1 [-1]
1 [0, 1]
-1 [-1]
-1 [-1]
-1 [0, 1]
0 [0, 1]
0 [0, 1]
-1 [-1]
-1 [-1]
-1 [-1]
-1 [-1]
0 [0, 1, 2]
0 [0, 1]
-1 [-1]
-1 [-1]
0 [0, 1]
-1 [-1]
-1 [-1]
1 [0, 1]
0 [0, 1, 2]
-1 [-1]
0 [0, 1]
-1 [-1]
-1 [-1]
-1 [-1]
-1 [-1]
0 [0, 1]
-1 [-1]
-1 [-1]
0 [0, 1]
-1 [-1]
1 [0, 1, 2, 3, 4]
0 [0, 1]
0 [0, 1]
-1 [-1]
-1 [-1]
1 [0, 1]
0 [0, 1]
0 [0, 1, 2, 3]
0 [0, 1]
0 [0, 1, 2]
1 [0, 1]
-1 [0, 1, 2]
0 [0, 1]
0 [0, 1]
-1 [0, 1]
-1 [-1]
1 [0, 1]
-1 [-1]
-1 [-1]
-1 [0, 1]
0 [0, 1]
0 [0, 1]
-1 [-1]
-1 [-1]
-1 [-1]
-1 [-1]
0 [0, 1, 2]
0 [0, 1]
-1 [-1]
-1 [-1]
0 [0, 1]
-1 [-1]
-1 [-1]
1 [0, 1]
0 